In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, mean_squared_error;
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV;
from sklearn.preprocessing import OneHotEncoder;
from sklearn.feature_extraction.text import CountVectorizer;
import gc;
import pickle;
import random;
import matplotlib.pyplot as plt;
import time;
import xlearn as xl

In [3]:
ratings = pd.read_csv("./ratings.csv")

ratings.columns = ['userId','movieId','rating','timestamp']
ratings = ratings.drop('timestamp', axis=1)
ratings

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5
...,...,...,...
27753439,283228,8542,4.5
27753440,283228,8712,4.5
27753441,283228,34405,4.5
27753442,283228,44761,4.5


In [4]:
y = ratings['rating'].values
X = ratings.drop('rating', axis=1)

In [5]:
y

array([3.5, 3.5, 1.5, ..., 4.5, 4.5, 4.5])

In [6]:
X

,userId,movieId
0,1,307
1,1,481
2,1,1091
3,1,1257
4,1,1449
...,...,...
27753439,283228,8542
27753440,283228,8712
27753441,283228,34405
27753442,283228,44761


In [7]:
encoder = OneHotEncoder(handle_unknown='ignore')

In [8]:
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.20, random_state=1234)
#trainX, testX = encode(trainX,testX)

encoder.fit(trainX)



OneHotEncoder(handle_unknown='ignore')

In [9]:
trainX = encoder.transform(trainX)

In [10]:
testX = encoder.transform(testX)
print(testX)

  (0, 61341)	1.0
  (0, 283846)	1.0
  (1, 45578)	1.0
  (1, 283077)	1.0
  (2, 246677)	1.0
  (2, 285196)	1.0
  (3, 200879)	1.0
  (3, 282228)	1.0
  (4, 182676)	1.0
  (4, 305529)	1.0
  (5, 57146)	1.0
  (5, 282457)	1.0
  (6, 146972)	1.0
  (6, 288810)	1.0
  (7, 208847)	1.0
  (7, 285468)	1.0
  (8, 33168)	1.0
  (8, 282455)	1.0
  (9, 41181)	1.0
  (9, 285235)	1.0
  (10, 170121)	1.0
  (10, 297574)	1.0
  (11, 172596)	1.0
  (11, 282892)	1.0
  (12, 44065)	1.0
  :	:
  (5550676, 283251)	1.0
  (5550677, 20004)	1.0
  (5550677, 282463)	1.0
  (5550678, 109116)	1.0
  (5550678, 311119)	1.0
  (5550679, 184208)	1.0
  (5550679, 293902)	1.0
  (5550680, 259817)	1.0
  (5550680, 302390)	1.0
  (5550681, 597)	1.0
  (5550681, 298487)	1.0
  (5550682, 28504)	1.0
  (5550682, 282132)	1.0
  (5550683, 25360)	1.0
  (5550683, 282132)	1.0
  (5550684, 177291)	1.0
  (5550684, 289975)	1.0
  (5550685, 43123)	1.0
  (5550685, 292940)	1.0
  (5550686, 229782)	1.0
  (5550686, 282146)	1.0
  (5550687, 180233)	1.0
  (5550687, 283773)	1.0


In [10]:
from fastFM.mcmc import FMClassification, FMRegression
from fastFM import als;

In [ ]:
cv = ShuffleSplit(n_splits=3, test_size= int(0.2 * trainX.shape[0]), random_state = 2018)


estimator = als.FMRegression()

params = {
'n_iter' : np.arange(10,100,30),
'rank' :  np.arange(2,12,4),
'l2_reg_w': np.logspace(-6, -1, 3),
'l2_reg_V' : np.logspace(-6, -1, 3)
}

###Gridsearch over parameters
regressor = GridSearchCV(estimator=estimator , cv=cv, param_grid=params)
regressor.fit(trainX, trainY)
